# PyJQuants Quickstart

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/obichan117/pyjquants/blob/main/docs/examples/quickstart.ipynb)

This notebook demonstrates the basic usage of PyJQuants - a yfinance-style Python library for [J-Quants API](https://jpx.gitbook.io/j-quants-en).

## Tier Information

J-Quants API has different subscription tiers with varying access levels:

| Tier | Requests/min | Monthly Fee | Best For |
|------|-------------|-------------|----------|
| **Free** | 5 | ¥0 | Testing, learning |
| **Light** | 60 | ~¥1,650 | Personal projects |
| **Standard** | 120 | ~¥3,300 | Active trading |
| **Premium** | 500 | ~¥16,500 | Production systems |

This notebook is organized by tier:

- **Sections 1-6**: Work with **ALL tiers** (Free, Light, Standard, Premium)
- **Section 7+**: Require **Standard tier or higher**

## Contents

### All Tiers (Free/Light/Standard/Premium)
1. [Setup](#1-setup)
2. [Single Ticker](#2-single-ticker)
3. [Price History](#3-price-history)
4. [Multi-Ticker Download](#4-multi-ticker-download)
5. [Financial Statements](#5-financial-statements)
6. [Market Information](#6-market-information)

### Standard+ Tier Only
7. [Morning Session Prices](#7-morning-session-prices-standard)
8. [Dividends & Detailed Financials](#8-dividends--detailed-financials-standard)
9. [Market Indices (Nikkei 225)](#9-market-indices-nikkei-225-standard)
10. [Sector Classifications](#10-sector-classifications-standard)
11. [Short Selling & Margin Data](#11-short-selling--margin-data-standard)
12. [Derivatives](#12-derivatives-standard)

---
# Part 1: All Tiers (Free/Light/Standard/Premium)

The following sections work with **all subscription tiers**, including the free tier.

---

## 1. Setup

### Install PyJQuants

In [ ]:
# Install PyJQuants (auto-detects Colab)
try:
    import pyjquants
    print(f'PyJQuants already installed (v{pyjquants.__version__})')
except ImportError:
    print('Installing PyJQuants...')
    !pip install -q pyjquants
    print('Installed!')

### Configure API Key

You need a J-Quants account. [Sign up here](https://application.jpx-jquants.com/) (free tier available).

Get your API key from the [J-Quants dashboard](https://application.jpx-jquants.com/), then run the cell below.

**Option 1 (Recommended for Colab):** Use Colab's Secrets manager:
1. Click the key icon in the left sidebar
2. Add a secret named `JQUANTS_API_KEY` with your API key
3. Toggle "Notebook access" ON

**Option 2:** Paste your API key directly when prompted below.

In [ ]:
import os

# Try multiple sources for API key
api_key = None

# 1. Check environment variable (already set)
if os.environ.get('JQUANTS_API_KEY'):
    api_key = os.environ['JQUANTS_API_KEY']
    print('API key found in environment')

# 2. Try Colab secrets (if running in Colab)
if not api_key:
    try:
        from google.colab import userdata
        api_key = userdata.get('JQUANTS_API_KEY')
        os.environ['JQUANTS_API_KEY'] = api_key
        print('API key loaded from Colab Secrets')
    except:
        pass

# 3. Prompt user to paste API key
if not api_key:
    print('=' * 50)
    print('Paste your J-Quants API key below and press Enter:')
    print('(Get it from: https://application.jpx-jquants.com/)')
    print('=' * 50)
    api_key = input('API Key: ').strip()
    
    if api_key:
        os.environ['JQUANTS_API_KEY'] = api_key
        print('API key set!')
    else:
        print('No API key provided. Please set JQUANTS_API_KEY.')

# Verify
if os.environ.get('JQUANTS_API_KEY'):
    print(f"\nAPI key configured (ends with ...{os.environ['JQUANTS_API_KEY'][-4:]})")

### Import the Library

In [ ]:
import pyjquants as pjq

print(f'PyJQuants version: {pjq.__version__}')

## 2. Single Ticker

Create a `Ticker` object using the stock code. This follows the yfinance API style.

In [ ]:
# Create a ticker object (Toyota)
ticker = pjq.Ticker('7203')

# Access stock information (lazy-loaded from API)
print(f'Code: {ticker.info.code}')
print(f'Name: {ticker.info.name}')
print(f'English Name: {ticker.info.name_english}')
print(f'Sector: {ticker.info.sector}')
print(f'Market: {ticker.info.market}')

### Search for Tickers

In [ ]:
# Search by name
tickers = pjq.search('toyota')
print(f'Found {len(tickers)} tickers matching "toyota":')
for t in tickers[:5]:  # Show first 5
    print(f'  {t.code}: {t.info.name}')

## 3. Price History

Use the `.history()` method to get price data, just like yfinance.

### Recent Prices

In [ ]:
# Get recent 30 days of price data (default)
df = ticker.history('30d')

print(f'Data points: {len(df)}')
print(f'Columns: {list(df.columns)}')
print()
df[['date', 'open', 'high', 'low', 'close', 'volume']].tail()

### Different Time Periods

In [ ]:
# Various period formats
df_1y = ticker.history('1y')     # 1 year
df_6mo = ticker.history('6mo')   # 6 months
df_1w = ticker.history('1w')     # 1 week

print(f'1 year: {len(df_1y)} days')
print(f'6 months: {len(df_6mo)} days')
print(f'1 week: {len(df_1w)} days')

### Custom Date Range

In [ ]:
from datetime import date

# Get prices for a specific date range
df_custom = ticker.history(start='2024-01-01', end='2024-06-30')

# Or using date objects
df_custom = ticker.history(start=date(2024, 1, 1), end=date(2024, 6, 30))

print(f'Custom range: {len(df_custom)} trading days')
df_custom[['date', 'close']].head(10)

### Visualize Prices

In [ ]:
import matplotlib.pyplot as plt

# Get 1 year of data for visualization
df = ticker.history('1y')

fig, ax = plt.subplots(figsize=(12, 6))

ax.plot(df['date'], df['close'], 'b-', linewidth=2)
ax.set_title(f'{ticker.info.name} ({ticker.code}) - Stock Price', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Close Price (JPY)')
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 4. Multi-Ticker Download

Download price data for multiple tickers at once using `pjq.download()`.

In [ ]:
# Download close prices for multiple tickers
codes = ['7203', '6758', '7974', '9984']  # Toyota, Sony, Nintendo, SoftBank

df_multi = pjq.download(codes, period='30d')

print(f'Shape: {df_multi.shape}')
df_multi.tail()

In [ ]:
# Plot multiple stocks
fig, ax = plt.subplots(figsize=(12, 6))

for code in codes:
    ax.plot(df_multi['date'], df_multi[code], label=code, linewidth=2)

ax.set_title('Stock Price Comparison', fontsize=14)
ax.set_xlabel('Date')
ax.set_ylabel('Close Price (JPY)')
ax.legend()
ax.grid(True, alpha=0.3)

plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

## 5. Financial Statements

Access financial statement summaries (available on all tiers).

In [ ]:
# Get financial statements
financials = ticker.financials

print(f'Financial records: {len(financials)}')
if len(financials) > 0:
    # Show recent records
    financials[['disclosure_date', 'type_of_document', 'net_sales', 'operating_profit']].head()

## 6. Market Information

Access market utilities like trading calendar and investor trades.

### TOPIX Index (Free/Light)

In [ ]:
# Get TOPIX index (available on all tiers)
topix = pjq.Index.topix()
print(f'Index: {topix.name}')

# Get TOPIX history
df_topix = topix.history('30d')
print(f'\nRecent TOPIX prices:')
df_topix[['date', 'close']].tail()

### Trading Calendar

In [ ]:
from datetime import date

market = pjq.Market()

# Check trading days
test_date = date(2024, 12, 25)  # Christmas
print(f'Is {test_date} a trading day? {market.is_trading_day(test_date)}')

# Get next trading day
new_year = date(2025, 1, 1)
next_day = market.next_trading_day(new_year)
print(f'Next trading day after {new_year}: {next_day}')

### Earnings Calendar

In [ ]:
# Get earnings announcements for a date range
df_earnings = market.earnings_calendar(
    start=date(2024, 10, 1),
    end=date(2024, 10, 31)
)

print(f'Earnings announcements in Oct 2024: {len(df_earnings)}')
if len(df_earnings) > 0:
    df_earnings[['code', 'company_name', 'announcement_date']].head(10)

### Investor Trades (Market-wide)

In [ ]:
# Get market-wide trading by investor type
df_investors = market.investor_trades(
    start=date(2024, 1, 1),
    end=date(2024, 3, 31)
)

print(f'Investor trade records: {len(df_investors)}')
if len(df_investors) > 0:
    df_investors.head()

### Margin Interest

In [ ]:
# Get margin trading balances
df_margin = market.margin_interest(code='7203')

print(f'Margin interest records: {len(df_margin)}')
if len(df_margin) > 0:
    df_margin.head()

---
# Part 2: Standard+ Tier Only

The following sections require **Standard tier or higher**.

If you're on the Free or Light tier, these cells will return errors (403 Forbidden) or empty results.

---

## 7. Morning Session Prices (Standard+)

Get morning session (AM) prices only, which close at 11:30 JST.

In [ ]:
# Morning session prices (Standard+ tier)
try:
    df_am = ticker.history_am('30d')
    print(f'AM session data points: {len(df_am)}')
    df_am[['date', 'open', 'high', 'low', 'close', 'volume']].tail()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('This endpoint requires Standard tier or higher.')

## 8. Dividends & Detailed Financials (Standard+)

### Dividend History

In [ ]:
# Dividend history (Standard+ tier)
try:
    dividends = ticker.dividends
    print(f'Dividend records: {len(dividends)}')
    if len(dividends) > 0:
        display(dividends[['record_date', 'dividend_per_share', 'payment_date']].head())
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('This endpoint requires Standard tier or higher.')

### Detailed Financials (BS/PL/CF)

In [ ]:
# Detailed financials (Standard+ tier)
try:
    details = ticker.financial_details
    print(f'Financial detail records: {len(details)}')
    if len(details) > 0:
        details.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('This endpoint requires Standard tier or higher.')

## 9. Market Indices - Nikkei 225 (Standard+)

While TOPIX is available on all tiers, Nikkei 225 requires Standard+.

In [ ]:
# Nikkei 225 index (Standard+ tier)
try:
    nikkei = pjq.Index.nikkei225()
    df_nikkei = nikkei.history('30d')
    print(f'Nikkei 225 data points: {len(df_nikkei)}')
    print(f'Latest close: {df_nikkei["close"].iloc[-1]:,.0f}')
    df_nikkei[['date', 'close']].tail()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('Nikkei 225 requires Standard tier or higher.')

## 10. Sector Classifications (Standard+)

TOPIX sector classifications (17 and 33 sectors).

In [ ]:
# TOPIX-17 sectors (Standard+ tier)
try:
    sectors_17 = market.sectors_17
    if len(sectors_17) > 0:
        print(f'TOPIX-17 sectors: {len(sectors_17)}')
        for s in sectors_17[:5]:
            print(f'  {s.code}: {s.name}')
    else:
        print('No data returned (tier restriction - Standard+ required)')
except Exception as e:
    print(f'Error: {e}')

In [ ]:
# TOPIX-33 sectors (Standard+ tier)
try:
    sectors_33 = market.sectors_33
    if len(sectors_33) > 0:
        print(f'TOPIX-33 sectors: {len(sectors_33)}')
        for s in sectors_33[:10]:
            print(f'  {s.code}: {s.name}')
    else:
        print('No data returned (tier restriction - Standard+ required)')
except Exception as e:
    print(f'Error: {e}')

## 11. Short Selling & Margin Data (Standard+)

### Trade Breakdown

In [ ]:
# Trade breakdown by type (Standard+ tier)
try:
    df_breakdown = market.breakdown('7203')
    print(f'Breakdown records: {len(df_breakdown)}')
    if len(df_breakdown) > 0:
        df_breakdown.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('This endpoint requires Standard tier or higher.')

### Short Selling Ratio

In [ ]:
# Short selling ratio (Standard+ tier)
try:
    df_short = market.short_ratio()
    print(f'Short ratio records: {len(df_short)}')
    if len(df_short) > 0:
        df_short.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('This endpoint requires Standard tier or higher.')

### Outstanding Short Positions

In [ ]:
# Outstanding short positions (Standard+ tier)
try:
    df_positions = market.short_positions()
    print(f'Short position records: {len(df_positions)}')
    if len(df_positions) > 0:
        df_positions.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('This endpoint requires Standard tier or higher.')

### Margin Trading Alerts

In [ ]:
# Margin trading alerts (Standard+ tier)
try:
    df_alerts = market.margin_alerts()
    print(f'Margin alert records: {len(df_alerts)}')
    if len(df_alerts) > 0:
        df_alerts.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('This endpoint requires Standard tier or higher.')

## 12. Derivatives (Standard+)

Futures and options data.

### Futures

In [ ]:
# Nikkei 225 mini futures (Standard+ tier)
try:
    futures = pjq.Futures('NK225M')
    df_futures = futures.history('30d')
    print(f'Futures data points: {len(df_futures)}')
    if len(df_futures) > 0:
        df_futures.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('Futures data requires Standard tier or higher.')

### Options

In [ ]:
# Options (Standard+ tier)
try:
    options = pjq.Options('NK225C25000')
    df_options = options.history('30d')
    print(f'Options data points: {len(df_options)}')
    if len(df_options) > 0:
        df_options.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('Options data requires Standard tier or higher.')

### Nikkei 225 Index Options

In [ ]:
# Nikkei 225 index options (Standard+ tier)
try:
    idx_opts = pjq.IndexOptions.nikkei225()
    df_idx_opts = idx_opts.history('30d')
    print(f'Index options data points: {len(df_idx_opts)}')
    if len(df_idx_opts) > 0:
        df_idx_opts.head()
except Exception as e:
    print(f'Error (likely tier restriction): {e}')
    print('Index options data requires Standard tier or higher.')

---
## Summary

### What's Available by Tier

| Feature | Free/Light | Standard+ |
|---------|:----------:|:---------:|
| Daily prices | Yes | Yes |
| Stock info & search | Yes | Yes |
| Financial statements | Yes | Yes |
| TOPIX index | Yes | Yes |
| Trading calendar | Yes | Yes |
| Earnings calendar | Yes | Yes |
| Investor trades | Yes | Yes |
| Margin interest | Yes | Yes |
| Morning session (AM) prices | - | Yes |
| Dividends | - | Yes |
| Detailed financials | - | Yes |
| Nikkei 225 index | - | Yes |
| Sector classifications | - | Yes |
| Trade breakdown | - | Yes |
| Short selling data | - | Yes |
| Margin alerts | - | Yes |
| Futures | - | Yes |
| Options | - | Yes |

### Resources

- [PyJQuants Documentation](https://obichan117.github.io/pyjquants)
- [GitHub Repository](https://github.com/obichan117/pyjquants)
- [J-Quants API Documentation](https://jpx.gitbook.io/j-quants-en/)
- [J-Quants Pricing](https://jpx-jquants.com/)